In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

fig_width = 960
fig_write = True

In [2]:
coureurs = pd.read_excel("acwe2023.xlsx", index_col="id", sheet_name="Coureurs")
motor = pd.read_excel("acwe2023.xlsx", index_col="id", sheet_name="Motor")
chassis = pd.read_excel("acwe2023.xlsx", index_col="id", sheet_name="Chassis")
teams = pd.read_excel("acwe2023.xlsx", index_col="id", sheet_name="Teams")

races = list(coureurs.columns[4:])

In [3]:
def calc_score_coureur(i, race):
    score = coureurs.loc[coureurs.index == i][race] * 0.5
    return score.values[0]

def calc_score_chassis(i, race):
    score = coureurs.loc[coureurs['chassis'] == i][race]
    return score.sum() * 0.2
    
def calc_score_motor(i, race):
    score = coureurs.loc[coureurs['motor'] == i][race]
    return score.sum() / (0.5*score.count()) * 0.3

In [4]:
df = pd.DataFrame(columns=['team', 'race', 'cat', 'score'])

for race in races:
    score_coureur1 = pd.Series([ calc_score_coureur(i, race) for i in teams['coureur1'] ])
    score_coureur2 = pd.Series([ calc_score_coureur(i, race) for i in teams['coureur2'] ])
    score_chassis  = pd.Series([ calc_score_chassis(i, race) for i in teams['chassis' ] ])
    score_motor    = pd.Series([ calc_score_motor  (i, race) for i in teams['motor'   ] ])
    
    df_coureur1 = pd.DataFrame({'team' : teams['naam'], 'race' : race, 'cat' : 'coureur1', 'score' : score_coureur1})
    df_coureur2 = pd.DataFrame({'team' : teams['naam'], 'race' : race, 'cat' : 'coureur2', 'score' : score_coureur2})
    df_chassis = pd.DataFrame({'team' : teams['naam'], 'race' : race, 'cat' : 'chassis', 'score' : score_chassis})
    df_motor = pd.DataFrame({'team' : teams['naam'], 'race' : race, 'cat' : 'motor', 'score' : score_motor})
    
    df = pd.concat([df, df_coureur1, df_coureur2, df_chassis, df_motor])

In [31]:
df_bahrein = df.loc[df['race']=='Bahrein']
df_bahrein = df_bahrein.sort_values(by="team")
fig = px.histogram(df_bahrein, x="team", y="score", color="cat",
            title="Resultaten Bahrein",
            width=fig_width, height=400,
            labels={ "score" : "Score", "cat" : "Categorie", "team" : "Teams" },
            template="simple_white")

fig.update_yaxes(showgrid=True)
fig.update_layout(yaxis_title="Punten")

series_name = ["Coureur 1", "Coureur 2", "Chassis", "Motor"]
for idx, name in enumerate(series_name):
    fig.data[idx].name = name
    fig.data[idx].hovertemplate=name + "<br>Punten = %{y}<extra></extra>"
#    fig.data[idx].hovertemplate = name

if fig_write:
    fig.write_html("resultaten_bahrein.html")
fig.show()

In [6]:
df.loc[df['race']=='Bahrein']

,team,race,cat,score
0,Chiel,Bahrein,coureur1,7.50
1,Marco,Bahrein,coureur1,3.00
2,Corine,Bahrein,coureur1,9.00
3,Casper,Bahrein,coureur1,5.00
4,Raymond,Bahrein,coureur1,9.00
...,...,...,...,...
17,Erwin,Bahrein,motor,6.45
18,Mees,Bahrein,motor,3.00
19,Jordi,Bahrein,motor,1.60
20,Wietse,Bahrein,motor,0.60


In [7]:
df_score_per_race = df.groupby(['race', 'team'])['score'].sum(numeric_only=True).unstack()#.reset_index()#.rename_axis(None, axis=1)
# df_score_per_race.transpose()
df_score_per_race = df_score_per_race.reindex(races)
df_score_cumsum = df_score_per_race.cumsum()

In [8]:
discrete_colors = px.colors.sample_colorscale('Bluered', np.arange(0.5, 21.5) / 22)
print(discrete_colors)
fig = px.line(
    df_score_cumsum, template='simple_white',
    title="Stand",
    labels={ "team" : "Teams" },
    width=fig_width, height=660)#, color_discrete_map=discrete_colors)
fig.update_layout(xaxis_title="Races") 
fig.update_layout(yaxis_title="Totale punten")
fig.update_yaxes(showgrid=True)

if fig_write:
    fig.write_html("totale_punten_lijn.html")
fig.show()

['rgb(6, 0, 249)', 'rgb(17, 0, 238)', 'rgb(29, 0, 226)', 'rgb(41, 0, 214)', 'rgb(52, 0, 203)', 'rgb(64, 0, 191)', 'rgb(75, 0, 180)', 'rgb(87, 0, 168)', 'rgb(99, 0, 156)', 'rgb(110, 0, 145)', 'rgb(122, 0, 133)', 'rgb(133, 0, 122)', 'rgb(145, 0, 110)', 'rgb(156, 0, 99)', 'rgb(168, 0, 87)', 'rgb(180, 0, 75)', 'rgb(191, 0, 64)', 'rgb(203, 0, 52)', 'rgb(214, 0, 41)', 'rgb(226, 0, 29)', 'rgb(238, 0, 17)']


In [27]:
# df_total_score = df.groupby(['team', 'race'], as_index=False)['score'].sum()
df_total_score = df.groupby(['team'], as_index=False)['score'].sum()
# df_total_score = df_total_score.sort_values(by='score', ascending=False)
df_total_score = df_total_score.sort_values(by='team', ascending=True)
df_total_score

,team,score
0,Arjan T.,11.45
1,Arjan Z.,8.00
2,Casper,19.75
3,Charlotte,11.00
4,Chiel,21.70
5,Corine,20.55
6,Emily,9.00
7,Erik,17.05
8,Erwin,6.85
9,Grietje,11.60


In [28]:
team_names = df_total_score['team']

fig = px.histogram(df, x="team", y="score", color="race",
            title="Stand",
            width=fig_width, height=660,
            labels={ "race" : "Races" },
            template="simple_white"
            )

fig.update_layout(xaxis_title="Teams") 
fig.update_layout(yaxis_title="Totale punten")

fig.update_xaxes(categoryorder='array', categoryarray=team_names)
fig.update_yaxes(showgrid=True)

for idx, name in enumerate(races):
    fig.data[idx].name = name
    fig.data[idx].hovertemplate = name + "<br>Punten = %{y}<extra></extra>"       
                 
if fig_write:
    fig.write_html("totale_punten_staaf.html")
fig.show()